In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from pyomo.opt import SolverFactory, TerminationCondition
from src.analyses.value_function import ValueFunction
from src.analyses.convergence_analysis import HausdorffAnalyzer
import numpy as np
import matplotlib.pyplot as plt
from src.models.cz_model import CaoZavalaModel, CaoZavalaAlgo
from scipy.optimize import curve_fit
from NSPLIB.src.instances.FLECCS.src.pyomo_model.model import const_model
fleccs_obj = -4.946928843629e+08  # Replace with actual value
fleccs_sol = {'x_sorbent_total': 0.7571021770410511,'x_air_adsorb_max': 0.7567643358197031} 
fleccs_y_bound = {'x_sorbent_total': [0.0, 1.0],"x_air_adsorb_max": [0.0, 1.0]}

n_day = 7
week_diff = 52
sto_m = const_model(n_day=n_day, week_diff=week_diff)
m = CaoZavalaModel.from_sto_m(sto_m)
m.build()
v_f= ValueFunction(m,solver="scip")
y_dimension = {'x_sorbent_total':{}, 'x_air_adsorb_max':{}}

for y in fleccs_y_bound:
    v_f.calc_1D(idx=y, y_val_fix=fleccs_sol,
                    interval= fleccs_y_bound[y], step = 3)
    for s in m.scenarios:
        y_dimension[y][s] = v_f.value_func[s]
    y_dimension[y]['total'] = v_f.total_value_func


# Specify grid dimensions
n_rows, n_cols = len(y_dimension), len(m.scenarios) + 1

# Create a grid of Output widgets for the plots
outputs = [[widgets.Output() for _ in range(n_cols)] for _ in range(n_rows)]

# Example: fill each plot with random data (replace with your actual data as needed)
for i in range(n_rows):
    for j in range(n_cols):
        with outputs[i][j]:
            x = np.linspace(0, 1, 3)
            y = y_dimension[list(y_dimension.keys())[i]][list(m.scenarios.keys())[j]]
            ax.plot(x, y)
            ax.set_title(f"Plot {i*n_cols + j + 1}")
            plt.show()

# Arrange the outputs in a grid
grid = widgets.GridBox(
    [outputs[i][j] for i in range(n_rows) for j in range(n_cols)],
    layout=widgets.Layout(grid_template_columns=f"repeat({n_cols}, 400px)")
)

display(grid)

In [6]:
import bqplot as bq
import ipywidgets as widgets
import numpy as np

# Create a grid of Output widgets
outputs = [[widgets.Output() for _ in range(2)] for _ in range(2)]

for i in range(2):
    for j in range(2):
        with outputs[i][j]:
            x_sc = bq.LinearScale()
            y_sc = bq.LinearScale()
            x = np.arange(10)
            y = np.random.rand(10)
            line = bq.Lines(x=x, y=y, scales={'x': x_sc, 'y': y_sc})
            ax_x = bq.Axis(scale=x_sc, label='x')
            ax_y = bq.Axis(scale=y_sc, orientation='vertical', label='y')
            fig = bq.Figure(marks=[line], axes=[ax_x, ax_y], title=f"Plot {i*2 + j + 1}")
            display(fig)

# Arrange the outputs in a grid
grid = widgets.GridBox(
    [outputs[i][j] for i in range(2) for j in range(2)],
    layout=widgets.Layout(grid_template_columns="repeat(2, 400px)")
)

display(grid)

GridBox(children=(Output(), Output(), Output(), Output()), layout=Layout(grid_template_columns='repeat(2, 400p…